# Handling Imbalanced Datasets

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.python.keras import layers, models
from sklearn.metrics import confusion_matrix , classification_report

# Data

In [7]:
df = pd.read_csv("Data/customer_churn.csv")
df.drop(['customerID'], axis=1, inplace=True)
df = df[df.TotalCharges!=' ']
df.TotalCharges = pd.to_numeric(df.TotalCharges)
df.replace('No internet service', 'No', inplace=True)
df.replace('No phone service', 'No', inplace=True)
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']

for col in yes_no_columns:
    df[col].replace({'Yes': 1, 'No': 0}, inplace=True)
df['gender'].replace({'Male': 1, 'Female': 0}, inplace=True)
df['gender'].unique()
df = pd.get_dummies(df, columns=['InternetService', 'Contract', 'PaymentMethod'])
col_to_scale = ['tenure', 'TotalCharges', 'MonthlyCharges']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[col_to_scale] = scaler.fit_transform(df[col_to_scale])

# Train/Test Split

In [8]:
X = df.drop('Churn', axis=1)
y = df['Churn']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Neural Network

In [9]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = models.Sequential([
        layers.Dense(26, input_dim=26, activation='relu'),
        layers.Dense(15, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    if weights == -1:
        model.fit(X_train, y_train, epochs=100)
    else:
        model.fit(X_train, y_train, epochs=100, class_weight = weights)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [10]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Metal device set to: Apple M1 Pro


2023-03-19 15:33:20.770942: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-19 15:33:20.770977: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-19 15:33:21.548569: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2023-03-19 15:33:21.970685: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


176/176 [==============================] - 4s 7ms/step - loss: 0.5298 - accuracy: 0.7188
Epoch 2/100
176/176 [==============================] - 1s 6ms/step - loss: 0.4336 - accuracy: 0.7934
Epoch 3/100
176/176 [==============================] - 1s 6ms/step - loss: 0.4240 - accuracy: 0.7979
Epoch 4/100
176/176 [==============================] - 1s 6ms/step - loss: 0.4189 - accuracy: 0.8012
Epoch 5/100
176/176 [==============================] - 1s 6ms/step - loss: 0.4175 - accuracy: 0.8034
Epoch 6/100
176/176 [==============================] - 1s 6ms/step - loss: 0.4148 - accuracy: 0.8053
Epoch 7/100
176/176 [==============================] - 1s 6ms/step - loss: 0.4147 - accuracy: 0.8064
Epoch 8/100
176/176 [==============================] - 1s 6ms/step - loss: 0.4126 - accuracy: 0.8071
Epoch 9/100
176/176 [==============================] - 1s 6ms/step - loss: 0.4120 - accuracy: 0.8068
Epoch 10/100
176/176 [==============================] - 1s 6ms/step - loss: 0.4110 - accuracy: 0.8052
E

2023-03-19 15:35:07.057418: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


44/44 [==============================] - 0s 4ms/step - loss: 0.4707 - accuracy: 0.7854
[0.47070130705833435, 0.7853589653968811]
Classification Report: 
               precision    recall  f1-score   support

           0       0.83      0.89      0.86      1038
           1       0.62      0.48      0.54       369

    accuracy                           0.79      1407
   macro avg       0.72      0.69      0.70      1407
weighted avg       0.77      0.79      0.78      1407



2023-03-19 15:35:07.374819: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


# Method 1: Undersampling

In [11]:
# Class count
count_class_0, count_class_1 = df.Churn.value_counts()

# Divide by class
df_class_0 = df[df['Churn'] == 0]
df_class_1 = df[df['Churn'] == 1]

# Undersample 0-class and concat the DataFrames of both class
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Churn.value_counts())

Random under-sampling:
0    1869
1    1869
Name: Churn, dtype: int64


In [12]:
X = df_test_under.drop('Churn',axis='columns')
y = df_test_under['Churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
y_pred = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
 1/94 [..............................] - ETA: 32s - loss: 0.7209 - accuracy: 0.5000

2023-03-19 15:41:38.616019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - 1s 11ms/step - loss: 0.6352 - accuracy: 0.6615
Epoch 2/100
94/94 [==============================] - 1s 6ms/step - loss: 0.5222 - accuracy: 0.7592
Epoch 3/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4916 - accuracy: 0.7686
Epoch 4/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4843 - accuracy: 0.7756
Epoch 5/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4781 - accuracy: 0.7779
Epoch 6/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4750 - accuracy: 0.7793
Epoch 7/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4708 - accuracy: 0.7789
Epoch 8/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4683 - accuracy: 0.7789
Epoch 9/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4652 - accuracy: 0.7833
Epoch 10/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4639 - accuracy: 0.7849
Epoch 11/100
94/94 [

2023-03-19 15:42:34.495024: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 0s 10ms/step - loss: 0.5873 - accuracy: 0.7059
[0.5872974395751953, 0.7058823704719543]
Classification Report: 
               precision    recall  f1-score   support

           0       0.73      0.66      0.69       374
           1       0.69      0.75      0.72       374

    accuracy                           0.71       748
   macro avg       0.71      0.71      0.71       748
weighted avg       0.71      0.71      0.71       748



2023-03-19 15:42:34.821062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Our F1 score for labelling churn has increased to 0.72

# Method 2: Oversampling

In [13]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Churn.value_counts())

Random over-sampling:
0    5163
1    5163
Name: Churn, dtype: int64


In [14]:
X = df_test_over.drop('Churn',axis='columns')
y = df_test_over['Churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [15]:
# Number of classes in training Data
y_train.value_counts()

1    4130
0    4130
Name: Churn, dtype: int64

In [16]:
loss = tf.keras.losses.BinaryCrossentropy()
weights = -1
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
  8/259 [..............................] - ETA: 1s - loss: 0.7241 - accuracy: 0.4336  

2023-03-19 15:44:32.707215: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


259/259 [==============================] - 2s 7ms/step - loss: 0.5584 - accuracy: 0.7168
Epoch 2/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4868 - accuracy: 0.7648
Epoch 3/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4787 - accuracy: 0.7673
Epoch 4/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4749 - accuracy: 0.7689
Epoch 5/100
259/259 [==============================] - 1s 6ms/step - loss: 0.4706 - accuracy: 0.7729
Epoch 6/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4682 - accuracy: 0.7757
Epoch 7/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4655 - accuracy: 0.7766
Epoch 8/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4643 - accuracy: 0.7814
Epoch 9/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4629 - accuracy: 0.7754
Epoch 10/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4598 - accuracy: 0.7766
E

2023-03-19 15:47:06.488407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


65/65 [==============================] - 0s 4ms/step - loss: 0.4624 - accuracy: 0.7880
[0.462367981672287, 0.7879961133003235]
Classification Report: 
               precision    recall  f1-score   support

           0       0.81      0.75      0.78      1033
           1       0.77      0.83      0.80      1033

    accuracy                           0.79      2066
   macro avg       0.79      0.79      0.79      2066
weighted avg       0.79      0.79      0.79      2066



2023-03-19 15:47:06.863196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Our F1 Score for churn(1) improved to 0.8 from 0.53

# Method 3: SMOTE
Oversampling by producing synthetic samples

In [21]:
X = df.drop('Churn',axis='columns')
y = df['Churn']

from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [22]:
# Number of classes in training Data
y_train.value_counts()

1    4130
0    4130
Name: Churn, dtype: int64

In [23]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
  9/259 [>.............................] - ETA: 1s - loss: 0.6982 - accuracy: 0.5208  

2023-03-19 15:50:51.758174: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


259/259 [==============================] - 2s 6ms/step - loss: 0.5477 - accuracy: 0.7303
Epoch 2/100
259/259 [==============================] - 2s 7ms/step - loss: 0.4690 - accuracy: 0.7781
Epoch 3/100
259/259 [==============================] - 2s 7ms/step - loss: 0.4592 - accuracy: 0.7817
Epoch 4/100
259/259 [==============================] - 1s 6ms/step - loss: 0.4535 - accuracy: 0.7835
Epoch 5/100
259/259 [==============================] - 1s 6ms/step - loss: 0.4498 - accuracy: 0.7877
Epoch 6/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4457 - accuracy: 0.7902
Epoch 7/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4413 - accuracy: 0.7930
Epoch 8/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4395 - accuracy: 0.7994
Epoch 9/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4354 - accuracy: 0.7975
Epoch 10/100
259/259 [==============================] - 2s 6ms/step - loss: 0.4330 - accuracy: 0.8000
E

2023-03-19 15:53:26.985353: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


65/65 [==============================] - 0s 4ms/step - loss: 0.4359 - accuracy: 0.8045
[0.4359270930290222, 0.8044530749320984]
Classification Report: 
               precision    recall  f1-score   support

           0       0.84      0.75      0.79      1033
           1       0.77      0.86      0.82      1033

    accuracy                           0.80      2066
   macro avg       0.81      0.80      0.80      2066
weighted avg       0.81      0.80      0.80      2066



2023-03-19 15:53:27.361470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Our F1 score for churn(1) improved from 0.8 to 0.82

# Method 4: Use of Ensemble methods with undersampling

In [24]:
df.Churn.value_counts()
# Regain Original features and labels
X = df.drop('Churn',axis='columns')
y = df['Churn']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
y_train.value_counts()
df3 = X_train.copy()
df3['Churn'] = y_train
df3_class0 = df3[df3.Churn==0]
df3_class1 = df3[df3.Churn==1]

def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Churn', axis='columns')
    y_train = df_train.Churn
    return X_train, y_train
X_train, y_train = get_train_batch(df3_class0, df3_class1, 0, 1495)
y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
 8/94 [=>............................] - ETA: 0s - loss: 0.7144 - accuracy: 0.4688 

2023-03-19 15:59:22.252818: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - 1s 7ms/step - loss: 0.6405 - accuracy: 0.6455
Epoch 2/100
94/94 [==============================] - 1s 6ms/step - loss: 0.5305 - accuracy: 0.7525
Epoch 3/100
94/94 [==============================] - 1s 6ms/step - loss: 0.5016 - accuracy: 0.7599
Epoch 4/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4944 - accuracy: 0.7659
Epoch 5/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4875 - accuracy: 0.7676
Epoch 6/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4844 - accuracy: 0.7702
Epoch 7/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4825 - accuracy: 0.7689
Epoch 8/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4802 - accuracy: 0.7712
Epoch 9/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4772 - accuracy: 0.7779
Epoch 10/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4762 - accuracy: 0.7702
Epoch 11/100
94/94 [=

2023-03-19 16:00:16.676325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


44/44 [==============================] - 0s 4ms/step - loss: 0.5618 - accuracy: 0.7292
[0.5617837309837341, 0.7292110919952393]
Classification Report: 
               precision    recall  f1-score   support

           0       0.89      0.73      0.80      1033
           1       0.49      0.74      0.59       374

    accuracy                           0.73      1407
   macro avg       0.69      0.73      0.69      1407
weighted avg       0.78      0.73      0.74      1407



2023-03-19 16:00:16.980217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [26]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 1495, 2990)
y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
 1/94 [..............................] - ETA: 31s - loss: 0.6864 - accuracy: 0.5312

2023-03-19 16:03:09.301377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - 1s 7ms/step - loss: 0.6358 - accuracy: 0.6572
Epoch 2/100
94/94 [==============================] - 1s 6ms/step - loss: 0.5264 - accuracy: 0.7495
Epoch 3/100
94/94 [==============================] - 1s 6ms/step - loss: 0.5006 - accuracy: 0.7609
Epoch 4/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4912 - accuracy: 0.7612
Epoch 5/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4867 - accuracy: 0.7619
Epoch 6/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4819 - accuracy: 0.7652
Epoch 7/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4814 - accuracy: 0.7652
Epoch 8/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4761 - accuracy: 0.7692
Epoch 9/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4753 - accuracy: 0.7722
Epoch 10/100
94/94 [==============================] - 1s 6ms/step - loss: 0.4726 - accuracy: 0.7699
Epoch 11/100
94/94 [=

2023-03-19 16:04:05.058890: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


44/44 [==============================] - 1s 5ms/step - loss: 0.6515 - accuracy: 0.6823
[0.6515149474143982, 0.6823027729988098]
Classification Report: 
               precision    recall  f1-score   support

           0       0.90      0.64      0.75      1033
           1       0.45      0.80      0.57       374

    accuracy                           0.68      1407
   macro avg       0.67      0.72      0.66      1407
weighted avg       0.78      0.68      0.70      1407



2023-03-19 16:04:05.361964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [27]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 2990, 4130)
y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
 9/83 [==>...........................] - ETA: 0s - loss: 0.6864 - accuracy: 0.5208 

2023-03-19 16:04:05.707217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - 1s 10ms/step - loss: 0.6417 - accuracy: 0.6228
Epoch 2/100
83/83 [==============================] - 0s 6ms/step - loss: 0.5377 - accuracy: 0.7454
Epoch 3/100
83/83 [==============================] - 0s 6ms/step - loss: 0.4985 - accuracy: 0.7590
Epoch 4/100
83/83 [==============================] - 1s 6ms/step - loss: 0.4880 - accuracy: 0.7700
Epoch 5/100
83/83 [==============================] - 0s 6ms/step - loss: 0.4807 - accuracy: 0.7666
Epoch 6/100
83/83 [==============================] - 0s 6ms/step - loss: 0.4744 - accuracy: 0.7719
Epoch 7/100
83/83 [==============================] - 0s 6ms/step - loss: 0.4719 - accuracy: 0.7761
Epoch 8/100
83/83 [==============================] - 0s 6ms/step - loss: 0.4700 - accuracy: 0.7719
Epoch 9/100
83/83 [==============================] - 0s 6ms/step - loss: 0.4673 - accuracy: 0.7791
Epoch 10/100
83/83 [==============================] - 0s 6ms/step - loss: 0.4641 - accuracy: 0.7795
Epoch 11/100
83/83 [

2023-03-19 16:04:54.299882: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


44/44 [==============================] - 0s 4ms/step - loss: 0.6314 - accuracy: 0.6837
[0.6314128637313843, 0.6837242245674133]
Classification Report: 
               precision    recall  f1-score   support

           0       0.88      0.65      0.75      1033
           1       0.44      0.76      0.56       374

    accuracy                           0.68      1407
   macro avg       0.66      0.71      0.66      1407
weighted avg       0.77      0.68      0.70      1407



2023-03-19 16:04:54.595973: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [28]:
len(y_pred1)

1407

In [29]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones>1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [30]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.90      0.68      0.77      1033
           1       0.47      0.78      0.59       374

    accuracy                           0.71      1407
   macro avg       0.68      0.73      0.68      1407
weighted avg       0.78      0.71      0.72      1407



Our F1 score for minority class improved to 0.59. However this was the lowest for all the techniques used above